In [33]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta
from zipfile import ZipFile  ## https://pypi.org/project/zipfile36/
from filehandling import BatchProcess
import winsound
download_path = r"C:\Users\lisa.smith\Downloads"



## The Functions

In [34]:
def create_unique_IDs(user_initials):
    global batch_ID, timestamp,current_date_time,three_hr_delay, three_hr_one_sec_delay,date_newsourcecharge,date, user
    user = user_initials+'-'
    #curr_date = dt.today().strftime('%Y-%m-%d-')
    #currentTime = dt.today().strftime("%H-%M")
    current_date_time= dt.today().strftime('%Y-%m-%d %H:%M:%S')
    three_hr_delay=(dt.today() + timedelta(hours=3)).strftime('%Y-%m-%d %H:%M:%S')
    three_hr_one_sec_delay=(dt.today() + timedelta(hours=3,seconds=1)).strftime('%Y-%m-%d %H:%M:%S')
    timestamp = dt.now().timestamp()-1660000000
    batch_ID = user +dt.today().strftime('%Y-%m-%d-%H%M')
    date_newsourcecharge = dt.today().strftime('-%Y%m%d')
    date= dt.today().strftime('%Y%m%d-%H%M')
    return  batch_ID, user,current_date_time,timestamp, current_date_time,three_hr_delay, three_hr_one_sec_delay,date_newsourcecharge,date





In [35]:
create_unique_IDs('LS')
three_hr_delay, batch_ID

('2022-11-22 13:35:45', 'LS-2022-11-22-1035')

## Part 1 Read File

In [36]:
%%time
MB_folderpath = r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\Alteryx\Alteryx output"
#"S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\Alteryx\Alteryx output\Duplicate_Update.xlsx"
input_filename = r"\Duplicate_Update.xlsx"
data=pd.read_excel(MB_folderpath + input_filename)
data.head(3)

CPU times: total: 46.9 ms
Wall time: 393 ms


,PG_ID,VAT,Our_oracle_no,Oracle_name,NPC,ITEM_NUMBER,CHARGE_LINE_NO,MAX_CHARGE_LINENO,PRIMARY_UOM_CODE,B1_PRICE_EXVAT,...,BASE_PRICE,EFFECTIVE_DATE,START_DATE,END_DATE,Code,ITEM_CLASS,CHANNEL,Multi_desc,Count_Dup,Decide1
0,5727,0,15727,Frimley Health NHS Ward Beverage Trolley Ingre...,ACT317,ACT317,1,1,CA,32.45,...,27.04,2021-06-08 00:00:00,2017-01-20,NaN,Update,Category_Tower_10,e-Direct,Single,2,Select 1


## Batch Read files from a folder

In [63]:
%%time
#TO DO - a bug if 2nd Add file
# Alteryx input for UOM column needs to be aligned in Adds
#PRIMARY_UOM_CODE in Adds
#PRICING_UOM_CODE in other output files


Alteryx_output_folderpath = r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\Alteryx\Alteryx output"
T4_output_folderpath = r"\Tower 04"

for index, filename in enumerate(BatchProcess(Alteryx_output_folderpath+T4_output_folderpath + r"\Output*.xlsx")):
    print(index)
    print(filename)
    if index == 0:
        file_output = pd.read_excel(filename)
        if 'add'.lower() in filename.lower():
            Print("yay")
            file_output.rename(columns={'PRICING_UOM_CODE':'PRICING_UOM_CODE_empty'}, inplace=True)
            file_output.rename(columns={'PRIMARY_UOM_CODE':'PRICING_UOM_CODE'}, inplace=True)
        print(file_output.columns.values)    
    else:
        file2 = pd.read_excel(filename)
        if 'add'.lower() in filename.lower():
            file2.rename(columns={'PRICING_UOM_CODE':'PRICING_UOM_CODE_empty'}, inplace=True)
            file2.rename(columns={'PRIMARY_UOM_CODE':'PRICING_UOM_CODE'}, inplace=True)
            print("yay again")
        file_output=file_output.append(file2,ignore_index=True)
        print(file_output.columns.values)
    print(np.shape(file_output))

0
S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\Alteryx\Alteryx output\Tower 04\Output_Price_Changes.xlsx
['PG_ID' 'VAT' 'Our_oracle_no' 'Oracle_name' 'NPC' 'ITEM_NUMBER'
 'CHARGE_LINE_NO' 'MAX_CHARGE_LINENO' 'PRIMARY_UOM_CODE' 'B1_PRICE_EXVAT'
 'SOURCE_PRICE_LIST_ID' 'PRICELIST_NAME' 'SOURCE_PRICE_LIST_ITEM_ID'
 'ITEM_NUMBER2' 'PRICING_UOM_CODE' 'SOURCE_CHARGE_ID' 'LAST_UPDATED_ID'
 'PARENT_SOURCE_ID' 'BASE_PRICE' 'EFFECTIVE_DATE' 'START_DATE' 'END_DATE'
 'Code' 'ITEM_CLASS' 'CHANNEL' 'Multi_desc' 'Count_Dup' 'Decide1']
(115, 28)
1
S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\Alteryx\Alteryx output\Tower 04\Output_UOM_Issue.xlsx


<timed exec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


['PG_ID' 'VAT' 'Our_oracle_no' 'Oracle_name' 'NPC' 'ITEM_NUMBER'
 'CHARGE_LINE_NO' 'MAX_CHARGE_LINENO' 'PRIMARY_UOM_CODE' 'B1_PRICE_EXVAT'
 'SOURCE_PRICE_LIST_ID' 'PRICELIST_NAME' 'SOURCE_PRICE_LIST_ITEM_ID'
 'ITEM_NUMBER2' 'PRICING_UOM_CODE' 'SOURCE_CHARGE_ID' 'LAST_UPDATED_ID'
 'PARENT_SOURCE_ID' 'BASE_PRICE' 'EFFECTIVE_DATE' 'START_DATE' 'END_DATE'
 'Code' 'ITEM_CLASS' 'CHANNEL' 'Multi_desc' 'Count_Dup' 'Decide1'
 'PRIMARY UOM' 'B1_QTY' 'B1_PRICE' 'B2_QTY' 'B2_PRICE' 'B3_QTY' 'B3_PRICE'
 'B4_QTY' 'B4_PRICE' 'B5_QTY' 'B5_PRICE' 'Right_ITEM_NUMBER'
 'CHARGE_LINE_NUMBER' 'BASE_QTY' 'ITEM_CLASS_CODE' 'TIER1_MIN' 'TIER1_MAX'
 'AMOUNT1' 'TIER2_MIN' 'TIER2_MAX' 'AMOUNT2' 'TIER3_MIN' 'TIER3_MAX'
 'AMOUNT3' 'TIER4_MIN' 'TIER4_MAX' 'AMOUNT4' 'Choose1' 'Start_date1'
 'End_date1' 'B1 Price Ex VAT' 'DIFF' 'Decide' 'B2 Price Ex VAT'
 'B3 Price Ex VAT' 'B4 Price Ex VAT' 'B5 Price Ex VAT']
(115, 65)
2
S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_

<timed exec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


['PG_ID' 'VAT' 'Our_oracle_no' 'Oracle_name' 'NPC' 'ITEM_NUMBER'
 'CHARGE_LINE_NO' 'MAX_CHARGE_LINENO' 'PRIMARY_UOM_CODE' 'B1_PRICE_EXVAT'
 'SOURCE_PRICE_LIST_ID' 'PRICELIST_NAME' 'SOURCE_PRICE_LIST_ITEM_ID'
 'ITEM_NUMBER2' 'PRICING_UOM_CODE' 'SOURCE_CHARGE_ID' 'LAST_UPDATED_ID'
 'PARENT_SOURCE_ID' 'BASE_PRICE' 'EFFECTIVE_DATE' 'START_DATE' 'END_DATE'
 'Code' 'ITEM_CLASS' 'CHANNEL' 'Multi_desc' 'Count_Dup' 'Decide1'
 'PRIMARY UOM' 'B1_QTY' 'B1_PRICE' 'B2_QTY' 'B2_PRICE' 'B3_QTY' 'B3_PRICE'
 'B4_QTY' 'B4_PRICE' 'B5_QTY' 'B5_PRICE' 'Right_ITEM_NUMBER'
 'CHARGE_LINE_NUMBER' 'BASE_QTY' 'ITEM_CLASS_CODE' 'TIER1_MIN' 'TIER1_MAX'
 'AMOUNT1' 'TIER2_MIN' 'TIER2_MAX' 'AMOUNT2' 'TIER3_MIN' 'TIER3_MAX'
 'AMOUNT3' 'TIER4_MIN' 'TIER4_MAX' 'AMOUNT4' 'Choose1' 'Start_date1'
 'End_date1' 'B1 Price Ex VAT' 'DIFF' 'Decide' 'B2 Price Ex VAT'
 'B3 Price Ex VAT' 'B4 Price Ex VAT' 'B5 Price Ex VAT' 'Charge_Line_No'
 'Max_Charge_LineNo' 'PRICING_UOM_CODE_empty' 'Channel']
(100115, 69)
CPU times: total: 40 s
W

## Sand-pit

In [64]:
audit_log_filename=r'fileoutput_c3p0_super_test'+ date +'.xlsx'
file_output.to_excel(audit_log_filename,index=False)

# Separate out dfs

In [65]:
#add unique params

#separate out dfs
dfs = [x for _, x in file_output.groupby('Code')]
#rename dfs according to filename or code

#loop through dfs to create sets of csvs

#concat csvs

## Loop through each dataframe to create a set of csv

In [66]:
%%time

#TO Do rearrange code, so that it adds unique params before split, so that auditlog can be created
count = 0
for df in dfs:
    #print(i)
    
    pricelists_lookup =df[['Oracle_name']]
    pricelist_unique=pricelists_lookup.drop_duplicates()
    pricelist_unique['PRICELIST_ID'] = np.arange(len(pricelist_unique))+round(timestamp*100)

    create_unique_IDs('LS')
    df['BATCH_NAME'] = batch_ID
    df['SOURCE_PRICELIST_ITEM_ID'] = np.arange(len(df))+round(timestamp*1000)
    df['SOURCE_CHARGE_ID'] = np.arange(len(df))+round(timestamp*10000)
    df['NEW_SOURCE_CHARGE_ID'] = ''
    df['PriceLists_OP_CODE'] = 'NO-OP'
    df['Items_OP_CODE'] = 'CREATE'
    df['Charges_OP_CODE'] = 'CREATE'
    df['CHARGE_LINE_NUMBER_new']=1
    df['START_DATE'] = three_hr_delay
    df['END_DATE'] = ''
    df['SOURCE_TIERED_PRICING_ID']='TH_'+df['SOURCE_CHARGE_ID'].astype(str)
    df['ITEM_LEVEL_CODE'] = 'ITEM'
    df['PRIMARY_PRICING_UOM'] ='Y'
    df['BASE_PRICE_new'] =df['B1_PRICE_EXVAT']

    df = df.assign(emptycol1= '',emptycol2= '',emptycol3= '',emptycol4= '',emptycol5= '',
    emptycol6= '',emptycol7= '',emptycol8= '', CAN_ADJUST_FLAG='Y',ENFORCE_ADJ_CALCULATION_FLAG='Y',
    LINE_TYPE_CODE='ORA_BUY',PARENT_ENTITY_TYPE_CODE='PRICE_LIST_ITEM',
    CHARGE_DEFINITION_CODE= 'QP_SALE_PRICE',CALCULATION_METHOD_CODE='PRICE',
    AGGREGATION_METHOD_CODE='ORA_ON_LINE',TIER_BASIS_TYPE_CODE='ORA_ITEM_QUANTITY',
    GRADUATED_CODE='HIGHEST_TIER',APPLICATION_METHOD_CODE='PER_UNIT',ADJUSTMENT_TYPE_CODE='DISCOUNT_AMOUNT',
    ADJUSTMENT_BASIS_NAME='BASE_PRICE')
    print(df['Code'].iloc[0])
    if df['Code'].iloc[0]=='Update':
        df['Items_OP_CODE'] = 'NO-OP'
        df['Charges_OP_CODE'] = 'END-INSERT'
        df['SOURCE_CHARGE_ID'] = np.where(df['LAST_UPDATED_ID']=='',df['SOURCE_CHARGE_ID'],df['LAST_UPDATED_ID'])
        df['NEW_SOURCE_CHARGE_ID'] = np.arange(len(df))+round(timestamp*10000)
        df['CHARGE_LINE_NUMBER_new']=df['CHARGE_LINE_NO']
        
    print(np.shape(df))
    
    
    data_joined = pd.merge(df,pricelist_unique,on='Oracle_name',how='inner')
    Pricelists_int = data_joined[['BATCH_NAME','PriceLists_OP_CODE','PRICELIST_ID','Oracle_name',]]
    Pricelists_int_unique=Pricelists_int.drop_duplicates(subset=['Oracle_name'])
    PriceListItems_int=data_joined[['Items_OP_CODE','PRICELIST_ID','SOURCE_PRICELIST_ITEM_ID','ITEM_LEVEL_CODE',
    'NPC','emptycol1','emptycol2','PRICING_UOM_CODE','PRIMARY_PRICING_UOM','LINE_TYPE_CODE']]
    PriceListCharges_int=data_joined[['Charges_OP_CODE','PRICELIST_ID','SOURCE_CHARGE_ID','NEW_SOURCE_CHARGE_ID',
    'PARENT_ENTITY_TYPE_CODE','SOURCE_PRICELIST_ITEM_ID','CHARGE_LINE_NUMBER_new','CHARGE_DEFINITION_CODE','emptycol1',
    'emptycol2', 'CALCULATION_METHOD_CODE','BASE_PRICE_new','emptycol3', 'emptycol4',
     'emptycol5','emptycol6','emptycol7','CAN_ADJUST_FLAG','START_DATE','END_DATE']]

    #append all the csv dfs
    if count == 0:
        Pricelists_int_unique_joined = Pricelists_int_unique
        PriceListItems_int_joined = PriceListItems_int
        PriceListCharges_int_joined=PriceListCharges_int
        print("0",df)
    else:
        Pricelists_int_unique_joined = pd.concat([Pricelists_int_unique_joined,Pricelists_int_unique])
        PriceListItems_int_joined = PriceListItems_int_joined.append(PriceListItems_int)
        PriceListCharges_int_joined=PriceListCharges_int_joined.append(PriceListCharges_int)
        print("1",df)
    #print('c',np.shape(pricelist_unique))
    #print('d',np.shape(data_joined))
    count+=1
    print("count",count)
    print('Pricelists:',np.shape(Pricelists_int_unique_joined))
    print('Items:',np.shape(PriceListItems_int_joined))
    print(PriceListItems_int_joined)
    print('Charges:',np.shape(PriceListCharges_int_joined))
    
headers_output_file = 'PriceListsHeaders.csv'
Pricelists_int_unique_joined.to_csv(headers_output_file,header=False,index=False)
items_output_file = 'PriceListsItems.csv'
PriceListItems_int_joined.to_csv(items_output_file,header=False,index=False)
charges_output_file = 'PriceListsCharges.csv'
PriceListCharges_int_joined.to_csv(charges_output_file,header=False,index=False)

date= dt.today().strftime('%Y%m%d_%H%M')

#Send to Zipped folder
zipObj = ZipFile(user+'c3po_super_test'+date+'.zip', 'w')
zipObj.write('PriceListsHeaders.csv')
zipObj.write('PriceListsItems.csv')
zipObj.write('PriceListsCharges.csv')

zipObj.close()

auditlog=data_joined[['BATCH_NAME','PRICELIST_ID','Oracle_name',
'SOURCE_PRICELIST_ITEM_ID','NPC','SOURCE_CHARGE_ID','NEW_SOURCE_CHARGE_ID',
'PRICING_UOM_CODE','CHARGE_LINE_NUMBER_new','BASE_PRICE_new','ITEM_CLASS','START_DATE','END_DATE']]

#auditlog_filepath=MB_folderpath+r'\03 Auditlog'
audit_log_filename=r'\auditlog_c3p0_super_test'+ date +'.xlsx'
auditlog.to_excel(download_path +audit_log_filename,index=False)



<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Update
(115, 100)
0     PG_ID VAT  Our_oracle_no            Oracle_name       NPC ITEM_NUMBER  \
0    5568  20        15568.0  ICF NCF Band 1 0 24 9  FQC22902    FQC22902   
1       1  20        10001.0               National   FJS4180     FJS4180   
2       1  20        10001.0               National   FJS4301     FJS4301   
3       1  20        10001.0               National   FJS4149     FJS4149   
4       1  20        10001.0               National   FJS4175     FJS4175   
..    ...  ..            ...                    ...       ...         ...   
110     1  20        10001.0               National   FJS4335     FJS4335   
111     1  20        10001.0               National   FJS4298     FJS4298   
112     1  20        10001.0               National   FJS4297     FJS4297   
113     1  20        10001.0               National   FJS4322     FJS4322   
114     1  20        10001.0               National   FJS4188     FJS4188   

     CHARGE_LINE_NO MAX_CHARGE_LINENO PRIMARY_UOM_CODE 

<timed exec>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


1        PG_ID VAT  Our_oracle_no  \
115     1347  20        11347.0   
116     2503  20        12503.0   
117     2503  20        12503.0   
118     2503  20        12503.0   
119     2503  20        12503.0   
...      ...  ..            ...   
100110  1781  20        11781.0   
100111  1121  20        11121.0   
100112   225  20        10225.0   
100113  1781  20        11781.0   
100114  2647  20        12647.0   

                                              Oracle_name       NPC  \
115                               Ophthalmology Mid Essex  ISY14974   
116         London North West Healthcare Nhs Trust Trauma   FJD9517   
117         London North West Healthcare Nhs Trust Trauma   FQN6297   
118         London North West Healthcare Nhs Trust Trauma   FJD8818   
119         London North West Healthcare Nhs Trust Trauma   FJK1973   
...                                                   ...       ...   
100110                             Barts Health Nhs Trust   FQT7815   
100111  N